# Paper

In [1]:
import tensorflow as tf
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pickle
import time
from xgboost import XGBClassifier
from xgboost import XGBRegressor
import torch.nn as nn
import tqdm
import torch
import sys
sys.path.append('/work/users/d/d/ddinh/aaco/src')
from load_dataset import load_adni_data
from cvar_sensing.utils import prepare_time_series, batch_interp_nd
import torch

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
from tensorflow.keras.metrics import AUC

2024-12-15 19:52:56.397040: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-15 19:52:56.410655: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-15 19:52:56.414797: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-15 19:52:56.426128: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-15 19:52:58.104055: W tensorflow/compiler/tf2

In [2]:
def load_data(file_path):
    ds = load_adni_data(file_path=file_path)
    x = ds.x
    y = ds.y
    mask_nan = np.isnan(x)
    x[mask_nan] = 0
    
    mask_nan_y = np.isnan(y)
    y[mask_nan_y] = 0
    return x, y

train_x, train_y = load_data("/work/users/d/d/ddinh/aaco/input_data/train_data.npz")
val_x, val_y = load_data("/work/users/d/d/ddinh/aaco/input_data/val_data.npz")
test_x, test_y = load_data("/work/users/d/d/ddinh/aaco/input_data/test_data.npz")

num_ts = train_x.shape[1]

In [3]:
def get_ts(x, y): 
    x_data = np.copy(x)
    y_data = np.copy(y)
    x_ts = []
    y_ts = []
    masks = []
    idx_table = []
    
    for i in range(num_ts):
        x_zero = np.zeros(x_data.shape)
        x_zero[:, :i+1] = x_data[:, :i+1,:]
        
        non_zero_mask = np.sum(y_data[:,i,:], axis=1) != 0
        
        x_filtered = np.transpose(x_zero[non_zero_mask,:,:], (0, 2, 1)).reshape(-1, x_data.shape[1] * x_data.shape[2])
        
        x_filtered = np.concatenate([x_filtered, np.repeat(i, x_filtered.shape[0])[:, None]], axis=1)
        
        x_ts.append(x_filtered)
        y_ts.append(y_data[non_zero_mask, i, :])
        # keep the index of the original data
        idx_table.append(np.where(non_zero_mask)[0])
            
    x_ts = np.concatenate(x_ts, axis=0)
    y_ts = np.concatenate(y_ts, axis=0)
    idx_table = np.concatenate(idx_table, axis=0)
    
    return x_ts, y_ts, idx_table

In [4]:
# train_x = np.transpose(train_x, (0, 2, 1)).reshape(-1, train_x.shape[1] * train_x.shape[2])
# val_x = np.transpose(val_x, (0, 2, 1)).reshape(-1, val_x.shape[1] * val_x.shape[2])
# test_x = np.transpose(test_x, (0, 2, 1)).reshape(-1, test_x.shape[1] * test_x.shape[2])

In [5]:
train_x_ts, train_y_ts, train_idx_table = get_ts(train_x, train_y)
val_x_ts, val_y_ts, val_idx_table = get_ts(val_x, val_y)
test_x_ts, test_y_ts, test_idx_table = get_ts(test_x, test_y)

# concatenate train and val
train_x_ts = np.concatenate([train_x_ts, val_x_ts], axis=0)
train_y_ts = np.concatenate([train_y_ts, val_y_ts], axis=0)
train_idx_table = np.concatenate([train_idx_table, val_idx_table], axis=0)

train_y_ts = np.argmax(train_y_ts, axis=1)  # shape (n,)
val_y_ts = np.argmax(val_y_ts, axis=1)  # shape (n,)
test_y_ts = np.argmax(test_y_ts, axis=1)  # shape (n,)

train_x_ts.shape, train_y_ts.shape, train_idx_table.shape

((6046, 49), (6046,), (6046,))

In [6]:
classifier = tf.keras.models.load_model('/work/users/d/d/ddinh/aaco/models/mlp.keras')

In [7]:
# todo: fix the build_pairs so the idx_table is used correclty

In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

# -------------------------------------------------------
# Hyperparameters
# -------------------------------------------------------
EMBED_DIM = 20         # Embedding dimension
NUM_CLASSES = 3
MARGIN = 1.0
LAMBDA_CONTRAST = 1.0
LAMBDA_CLASS = 1.0
BATCH_SIZE = 128
EPOCHS = 15
LEARNING_RATE = 1e-3

num_samples = train_x_ts.shape[0]
num_features = train_x_ts.shape[1]

X = train_x_ts
y = train_y_ts  # assumed shape (num_samples,); if it was (num_samples, 3), convert via np.argmax(...)

# Separate validation data
X_val = val_x_ts     # shape (num_val_samples, num_features)
y_val = val_y_ts     # shape (num_val_samples,)

def random_feature_mask(batch_x):
    """
    Takes a batch of shape (batch_size, num_features) 
    and applies random masks to the first (num_features-1) columns.
    """
    # Make a copy to avoid in-place modification of original data
    temp_x = batch_x.copy()
    
    # Build a mask for each row in the batch
    mask_list = []
    for _ in range(batch_x.shape[0]):
        # Randomly pick how many columns to sum from identity permutation
        n_cols = np.random.randint(1, num_features)  # or num_features-1
        # Permute (num_features-1) identity matrix, sum slice of columns
        mask_vec = np.sum(np.random.permutation(np.eye(num_features-1))[:, :n_cols-1], axis=1, keepdims=True)
        mask_list.append(mask_vec)

    # Concatenate into (batch_size, num_features-1)
    mask = np.concatenate(mask_list, axis=1).T.astype(np.float32)

    # Zero out mask where the original input was zero
    zero_input = temp_x[:, :num_features-1] == 0
    mask[zero_input] = 0
    
    temp_x[:, :num_features-1] *= mask
    return temp_x

def pair_generator(X, y, batch_size, num_features, p_same_sample=0.3, train_idx_table=None):
    """
    Yields ([batch_x1, batch_x2], [contrast_labels, class_labels]) one batch at a time.
    Re-samples the random mask *every iteration* (no pre-generation).
    """
    num_samples = X.shape[0]
    while True:
        batch_x1 = np.zeros((batch_size, num_features), dtype=np.float32)
        batch_x2 = np.zeros((batch_size, num_features), dtype=np.float32)
        batch_labels = np.zeros((batch_size,), dtype=np.float32)         
        batch_class_labels = np.zeros((batch_size,), dtype=np.int32)

        for i in range(batch_size):
            # Decide positive vs negative pair
            if i < batch_size // 2:
                # Positive pair
                if np.random.rand() < p_same_sample:
                    # Option A: same sample, different random masks
                    idx = np.random.randint(0, num_samples)
                    batch_x1[i] = random_feature_mask(X[idx:idx+1])
                    batch_x2[i] = random_feature_mask(X[idx:idx+1])
                    batch_labels[i] = 1.0
                    batch_class_labels[i] = y[idx]
                else:
                    # Option B: different samples, same class
                    while True:
                        idx1 = np.random.randint(0, num_samples)
                        idx2 = np.random.randint(0, num_samples)
                        if y[idx1] == y[idx2] and idx1 != idx2:
                            if train_idx_table is not None and (train_idx_table[idx1] == train_idx_table[idx2]):
                                continue
                            break
                    batch_x1[i] = random_feature_mask(X[idx1:idx1+1])
                    batch_x2[i] = random_feature_mask(X[idx2:idx2+1])
                    batch_labels[i] = 1.0
                    batch_class_labels[i] = y[idx1]
            else:
                # Negative pair
                while True:
                    idx1 = np.random.randint(0, num_samples)
                    idx2 = np.random.randint(0, num_samples)
                    if y[idx1] != y[idx2] and idx1 != idx2:
                        if train_idx_table is not None and (train_idx_table[idx1] == train_idx_table[idx2]):
                            continue
                        break
                batch_x1[i] = random_feature_mask(X[idx1:idx1+1])
                batch_x2[i] = random_feature_mask(X[idx2:idx2+1])
                batch_labels[i] = 0.0
                batch_class_labels[i] = y[idx1]

        yield ((batch_x1, batch_x2), (batch_labels, batch_class_labels))



# Create the training dataset with from_generator, infinite
train_dataset = tf.data.Dataset.from_generator(
    lambda: pair_generator(X, y, BATCH_SIZE, num_features, p_same_sample=0.3, train_idx_table=train_idx_table),
    output_signature = (
        (
            tf.TensorSpec(shape=(None, num_features), dtype=tf.float32),
            tf.TensorSpec(shape=(None, num_features), dtype=tf.float32),
        ),
        (
            tf.TensorSpec(shape=(None,), dtype=tf.float32),
            tf.TensorSpec(shape=(None,), dtype=tf.int32),
        )
    )
).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_generator(
    lambda: pair_generator(X_val, y_val, BATCH_SIZE, num_features, p_same_sample=0.3, train_idx_table=val_idx_table),
    output_signature = (
        (
            tf.TensorSpec(shape=(None, num_features), dtype=tf.float32),
            tf.TensorSpec(shape=(None, num_features), dtype=tf.float32),
        ),
        (
            tf.TensorSpec(shape=(None,), dtype=tf.float32),
            tf.TensorSpec(shape=(None,), dtype=tf.int32),
        )
    )
).prefetch(tf.data.AUTOTUNE)


# -------------------------------------------------------
# Embedding Network + Classification Head Combined
# -------------------------------------------------------

class L2NormalizeLayer(layers.Layer):
    def __init__(self, **kwargs):
        super(L2NormalizeLayer, self).__init__(**kwargs)

    def call(self, inputs):
        return tf.nn.l2_normalize(inputs, axis=1)

    def get_config(self):
        base_config = super(L2NormalizeLayer, self).get_config()
        return base_config

def create_embedding_and_classification(input_shape=(num_features,), embed_dim=EMBED_DIM, num_classes=NUM_CLASSES):
    """Returns a Keras model that outputs BOTH the embedding z and class logits."""
    inputs = keras.Input(shape=input_shape)
    x = layers.Dense(10, activation='relu')(inputs)
    x = layers.Dense(10, activation='relu')(x)
    z = layers.Dense(embed_dim, activation=None)(x)
    # L2 normalize via a Lambda layer
    z_norm = L2NormalizeLayer()(z)

    
    # Classification head on top of embedding
    class_logits = layers.Dense(num_classes, activation=None, name='class_logits')(z_norm)
    # We output the embedding (z_norm) and the class logits
    model = keras.Model(inputs, outputs=[z_norm, class_logits], name="EmbeddingPlusClass")
    return model

base_network = create_embedding_and_classification()

# -------------------------------------------------------
# Siamese Model
# -------------------------------------------------------
input_a = keras.Input(shape=(num_features,), name='input_a')
input_b = keras.Input(shape=(num_features,), name='input_b')

z1, class_logits_a = base_network(input_a)
z2, _ = base_network(input_b)

siamese_model = keras.Model(inputs=[input_a, input_b],
                            outputs=[z1, z2, class_logits_a],
                            name="AFA_Contrastive_Siamese")

# -------------------------------------------------------
# Contrastive Loss
# -------------------------------------------------------
def contrastive_loss(y_true, z1, z2, margin=MARGIN):
    distances = tf.reduce_sum(tf.square(z1 - z2), axis=1)
    positive_loss = y_true * distances
    negative_loss = (1 - y_true) * tf.square(tf.maximum(0.0, margin - tf.sqrt(distances + 1e-7)))
    return tf.reduce_mean(positive_loss + negative_loss)

# -------------------------------------------------------
# Custom Training (Subclassed) + Extra Metrics (ROC AUC, PR AUC)
# -------------------------------------------------------
class AFAContrastiveTrainer(keras.Model):
    def __init__(self, siamese_model, margin=MARGIN, lambda_contrast=1.0, lambda_class=1.0):
        super().__init__()
        self.siamese_model = siamese_model
        self.margin = margin
        self.lambda_contrast = lambda_contrast
        self.lambda_class = lambda_class

        # Main metrics
        self.loss_tracker = keras.metrics.Mean(name="loss")
        self.contrastive_loss_tracker = keras.metrics.Mean(name="contrastive_loss")
        self.class_loss_tracker = keras.metrics.Mean(name="class_loss")
        self.acc_metric = keras.metrics.SparseCategoricalAccuracy(name="accuracy")

        # Extra metrics: multi-label ROC AUC & PR AUC for a 3-class problem
        # We'll treat each class as separate label => multi_label=True, num_labels=NUM_CLASSES
        self.roc_auc = tf.keras.metrics.AUC(curve='ROC', multi_label=True, num_labels=NUM_CLASSES, name='roc_auc')
        self.pr_auc = tf.keras.metrics.AUC(curve='PR', multi_label=True, num_labels=NUM_CLASSES, name='pr_auc')

    def train_step(self, data):
        (x1, x2), (contrast_labels, class_labels) = data

        with tf.GradientTape() as tape:
            z1, z2, class_logits = self.siamese_model([x1, x2], training=True)
            # Contrastive loss
            c_loss = contrastive_loss(contrast_labels, z1, z2, margin=self.margin)
            # Classification loss
            class_loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
            cl_loss = class_loss_fn(class_labels, class_logits)
            total_loss = self.lambda_contrast * c_loss + self.lambda_class * cl_loss

        grads = tape.gradient(total_loss, self.siamese_model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.siamese_model.trainable_variables))

        # Update metrics
        self.loss_tracker.update_state(total_loss)
        self.contrastive_loss_tracker.update_state(c_loss)
        self.class_loss_tracker.update_state(cl_loss)
        self.acc_metric.update_state(class_labels, class_logits)

        # For ROC/PR AUC, we need probabilities and one-hot labels
        probs = tf.nn.softmax(class_logits, axis=1)
        labels_onehot = tf.one_hot(class_labels, depth=NUM_CLASSES)
        self.roc_auc.update_state(labels_onehot, probs)
        self.pr_auc.update_state(labels_onehot, probs)

        return {
            "loss": self.loss_tracker.result(),
            "contrastive_loss": self.contrastive_loss_tracker.result(),
            "class_loss": self.class_loss_tracker.result(),
            "accuracy": self.acc_metric.result(),
            "roc_auc": self.roc_auc.result(),
            "pr_auc": self.pr_auc.result(),
        }

    def test_step(self, data):
        (x1, x2), (contrast_labels, class_labels) = data
        z1, z2, class_logits = self.siamese_model([x1, x2], training=False)

        c_loss = contrastive_loss(contrast_labels, z1, z2, margin=self.margin)
        class_loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
        cl_loss = class_loss_fn(class_labels, class_logits)
        total_loss = self.lambda_contrast * c_loss + self.lambda_class * cl_loss

        self.loss_tracker.update_state(total_loss)
        self.contrastive_loss_tracker.update_state(c_loss)
        self.class_loss_tracker.update_state(cl_loss)
        self.acc_metric.update_state(class_labels, class_logits)

        # ROC/PR AUC
        probs = tf.nn.softmax(class_logits, axis=1)
        labels_onehot = tf.one_hot(class_labels, depth=NUM_CLASSES)
        self.roc_auc.update_state(labels_onehot, probs)
        self.pr_auc.update_state(labels_onehot, probs)

        return {
            "loss": self.loss_tracker.result(),
            "contrastive_loss": self.contrastive_loss_tracker.result(),
            "class_loss": self.class_loss_tracker.result(),
            "accuracy": self.acc_metric.result(),
            "roc_auc": self.roc_auc.result(),
            "pr_auc": self.pr_auc.result(),
        }

    @property
    def metrics(self):
        # Keras will reset each metric at the start of each epoch
        return [
            self.loss_tracker,
            self.contrastive_loss_tracker,
            self.class_loss_tracker,
            self.acc_metric,
            self.roc_auc,
            self.pr_auc,
        ]

# Instantiate and compile trainer
trainer = AFAContrastiveTrainer(
    siamese_model=siamese_model,
    margin=MARGIN,
    lambda_contrast=LAMBDA_CONTRAST,
    lambda_class=LAMBDA_CLASS
)
trainer.compile(optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE))

# -------------------------------------------------------
# Train with validation
# -------------------------------------------------------
trainer.fit(
    train_dataset,
    steps_per_epoch=100,
    epochs=EPOCHS,
    verbose=1,
    validation_data=val_dataset,
    validation_steps=30,   # example
)


Epoch 1/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.4513 - class_loss: 1.0463 - contrastive_loss: 0.3320 - loss: 1.3783 - pr_auc: 0.3426 - roc_auc: 0.5190 - val_accuracy: 0.5893 - val_class_loss: 0.9355 - val_contrastive_loss: 0.2633 - val_loss: 1.1988 - val_pr_auc: 0.3951 - val_roc_auc: 0.5946
Epoch 2/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.5650 - class_loss: 0.9377 - contrastive_loss: 0.2540 - loss: 1.1917 - pr_auc: 0.4193 - roc_auc: 0.6284 - val_accuracy: 0.5794 - val_class_loss: 0.9184 - val_contrastive_loss: 0.2471 - val_loss: 1.1656 - val_pr_auc: 0.4508 - val_roc_auc: 0.6390
Epoch 3/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.5675 - class_loss: 0.9105 - contrastive_loss: 0.2535 - loss: 1.1640 - pr_auc: 0.4605 - roc_auc: 0.6620 - val_accuracy: 0.5526 - val_class_loss: 0.9212 - val_contrastive_loss: 0.2453 - val_loss: 1.1665 - val_pr_auc: 0.4829 - val_roc_auc: 0.6601
Epoch 4/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.

In [11]:

base_network.save('/work/users/d/d/ddinh/aaco/models/embedding_classification_model.h5')


In [ ]:

# -------------------------------------------------------
# Inference / Usage
# -------------------------------------------------------
# After training, we can use the "base_network" to get both embeddings and classification.

x_new = np.random.randn(1, num_features).astype(np.float32)
x_new_masked = random_feature_mask(x_new, keep_prob=0.7)
z_new, class_logits_new = base_network.predict(x_new_masked)
print("Embedding shape:", z_new.shape)
print("Class logits shape:", class_logits_new.shape)
predicted_class = np.argmax(class_logits_new, axis=1)
print("Predicted class:", predicted_class)
